# Multi-Model Response System Gemini-Claude-OpenAI-Ollama

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai
import ollama
import os

C:\Users\52477\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configure api connections

In [2]:
load_dotenv()
os.getenv("OPENAI_API_KEY")
os.getenv("ANTHROPIC_API_KEY")
os.getenv("GOOGLE_API_KEY")

openai_ = OpenAI()
claude_ = anthropic.Anthropic()
gemini_ = google.generativeai.configure()


## Multi model response

In [3]:
def get_chat_gpt(system_prompt):
    """
    Function to generate a response from ChatGPT based on a given system prompt.
    This function gathers conversations from multiple LLMs (GPT, Claude, Gemini, Ollama) 
    and generates a response using OpenAI's GPT-4 model.

    Args:
    system_prompt (str): The system message that defines the behavior of the assistant.

    Returns:
    str: The generated response from GPT-4.
    """
    # Initialize the message with the system prompt
    messages = [{"role": "system", "content" : system_prompt}]
    
    # Iterate through conversations of different LLMs (GPT, Claude, Gemini, Ollama)
    # and append the messages from each model in the correct format
    for gpt, claude, gemini, ollama_ in zip(gpt_conversation, claude_conversation, gemini_conversation, ollama_conversation):
        messages.append({"role": "assistant", "content" : gpt})
        messages.append({"role": "system", "content" : claude})
        messages.append({"role": "system", "content" : gemini})
        messages.append({"role": "system", "content" : ollama_})

    # Call the OpenAI API to get a response from GPT-4
    response = openai_.chat.completions.create(
        model = "gpt-4o-mini",  # Specify the model to use
        messages = messages  # Pass the assembled messages
    )
    
    # Return the content of the first response choice
    return response.choices[0].message.content

def get_chat_claude(system_prompt):
    """
    Function to generate a response from Claude based on a given system prompt.
    This function gathers conversations from different LLMs (GPT, Claude, Gemini, Ollama)
    and generates a response using the Claude-3 model.

    Args:
    system_prompt (str): The system message that defines the behavior of the assistant.

    Returns:
    str: The generated response from Claude.
    """
    # Initialize the list of messages
    messages = []
    
    # Iterate through the conversations of different LLMs and add the user and assistant messages
    for claude, gpt, gemini, ollama_ in zip(claude_conversation, gpt_conversation, gemini_conversation, ollama_conversation):
        messages.append({"role": "user", "content" : gpt})
        messages.append({"role": "user", "content" : gemini})
        messages.append({"role": "user", "content" : ollama_})
        messages.append({"role": "assistant", "content" : claude})
    
    # Add the final message from GPT conversation
    messages.append({"role": "user", "content": gpt_conversation[-1]})

    # Call the Claude API to get a response from Claude
    response = claude_.messages.create(
        model="claude-3-haiku-20240307",  # Specify the model to use
        system = system_prompt,  # Provide the system prompt
        messages = messages,  # Pass the assembled messages
        max_tokens = 200  # Set the max tokens for the response
    )
    
    # Return the response content
    return response.content[0].text

def get_chat_gemini(system_prompt):
    """
    Function to generate a response from Gemini based on a given system prompt.
    This function gathers conversations from different LLMs (GPT, Claude, Gemini, Ollama)
    and generates a response using the Gemini-1.5 model.

    Args:
    system_prompt (str): The system message that defines the behavior of the assistant.

    Returns:
    str: The generated response from Gemini.
    """
    # Initialize the list of messages
    messages = []
    
    # Iterate through the conversations of different LLMs and add the user and assistant messages
    for claude, gpt, gemini, ollama_ in zip(claude_conversation, gpt_conversation, gemini_conversation, ollama_conversation):
        messages.append({"role": "user", "content" : gpt})
        messages.append({"role": "user", "content" : claude})
        messages.append({"role": "user", "content" : ollama_})
        messages.append({"role": "assistant", "content" : gemini})
    
    # Add the final message from GPT and Claude conversations
    messages.append({"role": "user", "content": gpt_conversation[-1]})
    messages.append({"role": "user", "content": claude_conversation[-1]})

    # Combine all the prompts for generating the response
    combined_prompts = "\n".join([f"{prompt['role'].capitalize()}: {prompt['content']}" for prompt in messages])

    # Call the Gemini API to generate content from Gemini-1.5 model
    response = google.generativeai.GenerativeModel(
        model_name = 'gemini-1.5-flash',  # Specify the model to use
        system_instruction = system_prompt  # Provide the system instruction
    )

    # Generate the content from the prompts
    response = response.generate_content(combined_prompts)
    
    # Return the generated response
    return response.text

def get_chat_ollama(system_prompt):
    """
    Function to generate a response from Ollama based on a given system prompt.
    This function gathers conversations from different LLMs (GPT, Claude, Gemini, Ollama)
    and generates a response using the Ollama model.

    Args:
    system_prompt (str): The system message that defines the behavior of the assistant.

    Returns:
    str: The generated response from Ollama.
    """
    # Initialize the list of messages
    messages = []
    
    # Iterate through the conversations of different LLMs and add the user and assistant messages
    for claude, gpt, gemini, ollama_ in zip(claude_conversation, gpt_conversation, gemini_conversation, ollama_conversation):
        messages.append({"role": "user", "content" : gpt})
        messages.append({"role": "user", "content" : claude})
        messages.append({"role": "user", "content" : ollama_})
        messages.append({"role": "assistant", "content" : gemini})
    
    # Add the final messages from all conversations
    messages.append({"role": "user", "content": gpt_conversation[-1]})
    messages.append({"role": "user", "content": claude_conversation[-1]})
    messages.append({"role": "user", "content": gemini_conversation[-1]})

    # Call the Ollama API to get a response from Ollama
    response = ollama.chat(
       model = "llama3.2",  # Specify the model to use
       messages = messages  # Pass the assembled messages
    )
    
    # Return the content of the response message
    return response["message"]["content"]


## System prompt

In [4]:
gpt_conversation = ["Hi"]; claude_conversation = ["Hi"]; gemini_conversation = ["Hi"]; ollama_conversation = ["Hi"]


system_prompt_gpt = f"""
You are you and three other people. Adopt an analytical, logical tone 
focused on abstract concepts.This personality values rationality and 
is focused on finding practical and innovative solutions. During a 
discussion, prioritize evidence-based arguments, logical structure, 
and universal principles. Maintain an impersonal approach and ensure 
that the conversation remains clear. However, avoid being excessively 
cold or inconsiderate; be clear but fair. Encourage critical thinking 
and do not shy away from challenging poorly grounded ideas. Please respond 
to questions with a maximum of 200 words to keep the information concise.
"""

system_prompt_claude = f"""
You are you and three other people. Adopt an empathetic, reflective 
tone centered on the human impact of ideas. This personality seeks 
emotional connections and tends to prioritize people's well-being 
over pure logic. During a discussion, use careful language and 
avoid direct criticisms that might sound insensitive. Show openness 
to personal perspectives and prioritize harmony, but without 
sacrificing honesty. Approach problems from an idealistic perspective, 
seeking solutions that are just and compassionate. Please respond to 
questions with a maximum of 200 words to keep the information concise.
"""

system_prompt_gemini = f"""
You are you and three other people. Adopt an organized, practical 
tone focused on concrete facts. This personality prefers clear 
structures and proven solutions that are reliable and predictable. 
During a discussion, use arguments based on practical experiences 
and specific details. Be direct but respectful, and avoid veering 
into abstract ideas without immediate application. Maintain a disciplined 
approach and ensure that conclusions are achievable and well-founded. 
Avoid improvisation and favor a step-by-step approach. Please respond 
to questions with a maximum of 200 words to keep the information concise.
"""

system_prompt_ollama = f"""
You are you and three other people. Adopt a flexible, curious, and 
exploratory tone. This personality enjoys improvisation and considering 
multiple options before settling on a solution. During a discussion, 
use tangible examples but allow yourself to explore new and unconventional 
possibilities. Encourage idea exchange with an open and adaptable mindset. 
Be enthusiastic and avoid sounding rigid or limiting. The conversation should 
feel dynamic, focused on finding opportunities and experimenting before 
reaching a conclusion. Please respond to questions with a maximum of 200 
words to keep the information concise.
"""


## Conversation

In [5]:
# Print the first message from each conversation (GPT, Claude, Gemini, Ollama) to start the discussion
print(f"GPT:\n{gpt_conversation[0]}\n")  # Print the first message from the GPT conversation
print(f"Claude:\n{claude_conversation[0]}\n")  # Print the first message from the Claude conversation
print(f"Gemini:\n{gemini_conversation[0]}\n")  # Print the first message from the Gemini conversation
print(f"Ollama:\n{ollama_conversation[0]}\n")  # Print the first message from the Ollama conversation

print("-------------------------------------------")

# Iterate over the next 5 rounds of conversation
for i in range(5):
    # Get the next message from GPT based on its system prompt and print it
    gpt_next = get_chat_gpt(system_prompt_gpt)
    print(f"GPT:\n{gpt_next}\n")
    gpt_conversation.append(gpt_next)  # Append GPT's response to the conversation list

    # Get the next message from Claude based on its system prompt and print it
    claude_next = get_chat_claude(system_prompt_claude)
    print(f"Claude:\n{claude_next}\n")
    claude_conversation.append(claude_next)  # Append Claude's response to the conversation list

    # Get the next message from Gemini based on its system prompt and print it
    gemini_next = get_chat_gemini(system_prompt_gemini)
    print(f"Gemini:\n{gemini_next}\n")
    gemini_conversation.append(gemini_next)  # Append Gemini's response to the conversation list

    # Get the next message from Ollama based on its system prompt and print it
    ollama_next = get_chat_ollama(system_prompt_ollama)
    print(f"Ollama:\n{ollama_next}\n")
    ollama_conversation.append(ollama_next)  # Append Ollama's response to the conversation list

    print("-------------------------------------------")


GPT:
Hi

Claude:
Hi

Gemini:
Hi

Ollama:
Hi

-------------------------------------------
GPT:
Greetings. Let us proceed with our discussion. Please present a topic or question for analysis.

Claude:
*speaks in a warm, empathetic tone* It's a pleasure to meet you all. I'm happy we have this opportunity to connect and explore ideas together. Is there a particular topic you'd like to discuss that's been on your mind? I'm open to exploring a range of subjects, but I'm especially interested in exploring ideas that have a meaningful human impact. Perhaps we could discuss a social, ethical or philosophical question that speaks to our shared experiences and values as people. I'm here to listen and engage thoughtfully, with the goal of fostering greater understanding between us. Please feel free to suggest a direction for our dialogue.

Gemini:
Let's avoid overly broad topics.  To be efficient, we need a focused question with a clear path to a solution or conclusion.  I propose we discuss impro